In [14]:
# Librería para comando de sistema
import os
# Librerías para manejo de datos
import pandas as pd
# Para realizar la separación del conjunto de aprendizaje en entrenamiento y test.
from sklearn.model_selection import train_test_split
# Para la creación de modelo Ridge
from sklearn.linear_model import Lasso
# Para búsqueda de hiperparámetros
from sklearn.model_selection import GridSearchCV
# Para la validación cruzada
from sklearn.model_selection import KFold 
# Para determinar el rendimiento del modelo con las métricas MSE, MAE y R2
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline

In [15]:
data = pd.read_excel("datos_proyecto_modulo2.xlsx")

In [16]:
# Es recomendable que todos los pasos de preparación se realicen sobre otro archivo.
data_t = data
# Eliminación data vacía.
data_t=data_t.dropna()
# Eliminación de registros duplicados.
data_t=data_t.drop_duplicates()
# Transformación de los datos categóricos a numéricos.

data_t = pd.get_dummies(data_t, columns=['clase_incendio', 'mes_incendio', 'vegetacion'], dtype=int)

In [17]:
# Se selecciona la variable objetivo, en este caso "precio".
Y=data_t['area_quemada']
# Del conjunto de datos se elimina la variable "precio".
X=data_t.drop(['area_quemada'], axis=1)

In [18]:
# Se realiza la división entrenamiento - test. Se deja 20% de los datos para el test.
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.20, random_state = 0)

In [19]:
# Para acelerar la convergencia del algoritmo que utiliza Lasso para optimizar la función de costo, utilizaremos la opción
# de normalizar los datos para que todos estén en el mismo rango.
modelo_lasso = make_pipeline(StandardScaler(), Lasso(alpha=1))
modelo_lasso

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('lasso', Lasso(alpha=1))])

In [20]:
# Ajuste del modelo
modelo_lasso.fit(X_train,Y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('lasso', Lasso(alpha=1))])

In [21]:
# Ahora probemos el rendimiento sobre el conjunto test.
y_pred = modelo_lasso.predict(X_test)
print("RMSE: %.2f" % mean_squared_error(Y_test, y_pred, squared=False))
print("MAE: %.2f" % mean_absolute_error(Y_test, y_pred))
print('R²: %.2f' % r2_score(Y_test, y_pred))

RMSE: 17241.30
MAE: 3644.48
R²: 0.23


c:\Users\harve\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Ahora vamos a utilizar la función GridSearchCV para la búsqueda del valor de alpha.

In [22]:
# Fijemos el número de particiones. Utilizaremos K = 10.
particiones = KFold(n_splits=10, shuffle=True, random_state = 0)

In [31]:
# Ahora tenemos que definir el espacio de búsqueda, es decir, los valores de alpha que queremos que sean considerados. 
# Para esto se define un diccionario (o grilla) con los valores que podrá asumir el hiperparámetro alpha.
# Probemos con los siguientes valores:
param_grid = {'lasso__alpha': [1, 2, 3, 4, 5]}

In [36]:
#definamoc el modelo lasso sin ningun alpha 
modelo_lasso = make_pipeline(StandardScaler(), Lasso())
modelo_lasso

Pipeline(steps=[('standardscaler', StandardScaler()), ('lasso', Lasso())])

In [37]:
# Ahora utilizamos GridSearch sobre el grid definido y con 10 particiones en la validación cruzada.
mejor_modelo = GridSearchCV(modelo_lasso, param_grid, cv=particiones, n_jobs=-1)
# Ajuste del modelo
mejor_modelo.fit(X_train, Y_train)

GridSearchCV(cv=KFold(n_splits=10, random_state=0, shuffle=True),
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('lasso', Lasso())]),
             n_jobs=-1, param_grid={'lasso__alpha': [1, 2, 3, 4, 5]})

In [38]:
# Podemos ver cual fue el resultado de la búsqueda (mejor valor de alpha)
print("Mejor parámetro: {}".format(mejor_modelo.best_params_))

Mejor parámetro: {'lasso__alpha': 5}


In [39]:
# También puedes indicarle a GridSearch que seleccione el mejor modelo a partir de la búsqueda con base en una métrica 
# particular. Por ejemplo, hubiésemos podido utilizar la siguiente línea de comando:
mejor_modelo = GridSearchCV(modelo_lasso, param_grid, scoring = 'neg_root_mean_squared_error', n_jobs=-1 )
mejor_modelo.fit(X_train, Y_train)
print("Mejor parámetro: {}".format(mejor_modelo.best_params_)) 

Mejor parámetro: {'lasso__alpha': 5}


In [40]:
modelo_final = mejor_modelo.best_estimator_
# Probemos ahora este modelo sobre test.
y_pred = modelo_final.predict(X_test)
print("RMSE: %.2f" % mean_squared_error(Y_test, y_pred, squared=False))
print("MAE: %.2f" % mean_absolute_error(Y_test, y_pred))
print('R²: %.2f' % r2_score(Y_test, y_pred))

RMSE: 17241.37
MAE: 3638.02
R²: 0.23


c:\Users\harve\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


5. Importancia de variables y selección
La regresión Lasso también actúa como un selector de variables: serán aquellas que tienen un coeficiente diferente de cero. Entonces, con base en los valores de los coeficientes, podrás determinar la importancia de los atributos en el analisis y predicción.

In [42]:
# Acceder al modelo Lasso dentro del Pipeline
modelo_lasso_entrenado = modelo_final.named_steps['lasso']

# Obtener los coeficientes
coeficientes = modelo_lasso_entrenado.coef_

# Obtener los nombres de las variables
variables = X_train.columns

# Mostrar en una tabla los valores de los coeficientes para cada variable

pd.DataFrame({'variables': variables, 'coeficientes': coeficientes})


,variables,coeficientes
0,Prec_pre_30,-19.738518
1,Prec_pre_15,-17.110006
2,Prec_pre_7,3.051377
3,Prec_cont,-45.769938
4,clase_incendio_B,-0.000000
5,clase_incendio_C,-15.275895
6,clase_incendio_D,-13.346564
7,clase_incendio_E,34.653529
8,clase_incendio_F,663.510707
9,clase_incendio_G,8140.142599
